In [3]:
!pip install lightgbm


  Using cached https://files.pythonhosted.org/packages/9c/b3/7b6aec53a8cd0a1378279d9f06e647298f0c66299bce0e4b2b919f6557f7/lightgbm-2.2.3-py2.py3-none-macosx_10_8_x86_64.macosx_10_9_x86_64.macosx_10_10_x86_64.macosx_10_11_x86_64.macosx_10_12_x86_64.macosx_10_13_x86_64.macosx_10_14_x86_64.whl


### Import Packages

In [1]:
import os
import pickle
import pymongo

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymongo import MongoClient
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

client=MongoClient('mongodb://localhost',27017)
print("Checking schema ... ")
print("")
print("The databse has the following databases:")

for db in client.database_names():
    print("Database: "+db)

print("")
print("The databse has the following collections:")
db=client.local
for coll in db.collection_names():
    print("Collection name: "+coll)


Checking schema ... 

The databse has the following databases:
Database: admin
Database: config
Database: local

The databse has the following collections:
Collection name: customer_chain
Collection name: startup_log
Collection name: customer_credits


/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.


### Data Extraction

In [21]:
train_raw=db.customer_chain.find()
credit_raw=db.customer_credits.find()

#change data rows into dataframe
new_df=pd.DataFrame([row for row in train_raw])
credit=pd.DataFrame([row for row in credit_raw])



In [22]:
new_df.head(10)



,Amount_Credit,Annuity,Birth_Day_CNT,Car_Age,Default,Employed_Day_CNT,Gender,ID,Income,Num_of_Child,Own_Car,Own_Residence,Price,Region_Population_Factor,Registration_Day_CNT,_id
0,756000.0,22234.5,-17106,,0,-8438,F,446165,247500.0,0,N,Y,756000,0.026392,-7502,5d579ed78f1998047d923f9d
1,450000.0,12375,-19454,,0,-899,F,150009,46566.0,0,N,N,450000,0.028663,-3224,5d579ed78f1998047d923f9e
2,295168.5,15201,-16139,,0,-2180,F,247486,76500.0,0,N,Y,238500,0.010556,-3410,5d579ed78f1998047d923f9f
3,900000.0,45954,-19430,,0,-805,M,164890,162000.0,0,N,N,900000,0.006008,-4366,5d579ed78f1998047d923fa0
4,445333.5,47961,-10681,5,0,-1921,M,353539,180000.0,1,Y,Y,423000,0.018801,-598,5d579ed78f1998047d923fa1
5,603000.0,24556.5,-11516,,0,-4165,F,304225,126000.0,0,N,N,603000,0.035792,-5522,5d579ed78f1998047d923fa2
6,713889.0,43803,-15666,5,0,365243,F,102316,270000.0,2,Y,Y,661500,0.072508,-3215,5d579ed78f1998047d923fa3
7,769500.0,27391.5,-14247,5,0,-2436,F,410579,247500.0,0,Y,Y,769500,0.026392,-8292,5d579ed78f1998047d923fa4
8,284400.0,13387.5,-10971,,0,-426,M,105812,144000.0,0,N,Y,225000,0.010147,-6267,5d579ed78f1998047d923fa5
9,284400.0,13387.5,-15615,1,0,-3459,F,127087,94500.0,0,Y,N,225000,0.031329,-4733,5d579ed78f1998047d923fa6


In [23]:
credit.head(10)



,Active_Credit,Credit,Credit_Days,Credit_Debt,Credit_Limit,Credit_Overdue,Currency,File_ID,ID,Max_Overdue,Overdue_Days,Type,_id
0,Closed,91323.00,-497,0,,0.0,currency 1,5714462,215354,,0,Consumer credit,5d579e3e8f1998047d913f7c
1,Active,90000.00,-203,,,0.0,currency 1,5714465,215354,,0,Credit card,5d579e3e8f1998047d913f7d
2,Active,225000.00,-208,171342,,0.0,currency 1,5714463,215354,,0,Credit card,5d579e3e8f1998047d913f7e
3,Active,464323.50,-203,,,0.0,currency 1,5714464,215354,,0,Consumer credit,5d579e3e8f1998047d913f7f
4,Active,2700000.00,-629,,,0.0,currency 1,5714466,215354,77674.5,0,Consumer credit,5d579e3e8f1998047d913f80
5,Active,180000.00,-273,71017.4,108983,0.0,currency 1,5714467,215354,0,0,Credit card,5d579e3e8f1998047d913f81
6,Active,42103.80,-43,42103.8,0,0.0,currency 1,5714468,215354,0,0,Consumer credit,5d579e3e8f1998047d913f82
7,Closed,76878.45,-1896,0,0,0.0,currency 1,5714469,162297,14985,0,Consumer credit,5d579e3e8f1998047d913f83
8,Closed,103007.70,-1146,0,0,0.0,currency 1,5714470,162297,0,0,Consumer credit,5d579e3e8f1998047d913f84
9,Active,4500.00,-1146,0,0,0.0,currency 1,5714471,162297,0,0,Credit card,5d579e3e8f1998047d913f85


In [24]:
new_df.shape



(65499, 16)

In [25]:
credit.shape



(65499, 13)

In [26]:
new_df.dtypes



Amount_Credit               float64
Annuity                      object
Birth_Day_CNT                 int64
Car_Age                      object
Default                       int64
Employed_Day_CNT              int64
Gender                       object
ID                            int64
Income                      float64
Num_of_Child                  int64
Own_Car                      object
Own_Residence                object
Price                        object
Region_Population_Factor    float64
Registration_Day_CNT          int64
_id                          object
dtype: object

### Data Cleasing


In [27]:
#Define function to change the unnormal type of variables sourced from data extraction to normal, here is object to string
def conv_obj(itm):
    try:
        return float(itm)
    except:
        return 0


In [28]:
# apply to correponding columns
new_df['Annuity']=new_df['Annuity'].apply(conv_obj)
new_df['Price']=new_df['Price'].apply(lambda x:conv_obj(x))


In [29]:
new_df.dtypes

Amount_Credit               float64
Annuity                     float64
Birth_Day_CNT                 int64
Car_Age                      object
Default                       int64
Employed_Day_CNT              int64
Gender                       object
ID                            int64
Income                      float64
Num_of_Child                  int64
Own_Car                      object
Own_Residence                object
Price                       float64
Region_Population_Factor    float64
Registration_Day_CNT          int64
_id                          object
dtype: object

In [37]:
client.close()
#delete unnormal value in "employed_day_cnt"
new_df['Employed_Day_CNT'] = (new_df['Employed_Day_CNT'].apply(lambda x:x if x !=365243 else np.nan))


In [31]:
#generate new columns with ratio to measure a person's ability for paying back  
new_df['Income/Credit'] = new_df['Income'] / new_df['Amount_Credit']

In [32]:
new_df['Annuity/Income'] = (new_df['Annuity'] / (1.0+new_df['Income']))**0.5

In [33]:
new_df['Payment_rate'] = new_df['Annuity'] / new_df['Amount_Credit']

In [34]:
new_df['Loan/Income'] = new_df['Amount_Credit'] / new_df['Income']

In [35]:
new_df['Credits/Goods'] = new_df['Amount_Credit'] / new_df['Price']

In [38]:
#drop "_id" column which is generated by mongodb system 
new_df=new_df.drop(columns=['_id'],axis=1)

In [39]:
new_df.head(10)



,Amount_Credit,Annuity,Birth_Day_CNT,Car_Age,Default,Employed_Day_CNT,Gender,ID,Income,Num_of_Child,Own_Car,Own_Residence,Price,Region_Population_Factor,Registration_Day_CNT,Income/Credit,Annuity/Income,Payment_rate,Loan/Income,Credits/Goods
0,756000.0,22234.5,-17106,,0,-8438.0,F,446165,247500.0,0,N,Y,756000.0,0.026392,-7502,0.327381,0.299727,0.029411,3.054545,1.000000
1,450000.0,12375.0,-19454,,0,-899.0,F,150009,46566.0,0,N,N,450000.0,0.028663,-3224,0.103480,0.515506,0.027500,9.663703,1.000000
2,295168.5,15201.0,-16139,,0,-2180.0,F,247486,76500.0,0,N,Y,238500.0,0.010556,-3410,0.259174,0.445761,0.051499,3.858412,1.237604
3,900000.0,45954.0,-19430,,0,-805.0,M,164890,162000.0,0,N,N,900000.0,0.006008,-4366,0.180000,0.532602,0.051060,5.555556,1.000000
4,445333.5,47961.0,-10681,5,0,-1921.0,M,353539,180000.0,1,Y,Y,423000.0,0.018801,-598,0.404191,0.516187,0.107697,2.474075,1.052798
5,603000.0,24556.5,-11516,,0,-4165.0,F,304225,126000.0,0,N,N,603000.0,0.035792,-5522,0.208955,0.441465,0.040724,4.785714,1.000000
6,713889.0,43803.0,-15666,5,0,NaN,F,102316,270000.0,2,Y,Y,661500.0,0.072508,-3215,0.378210,0.402781,0.061358,2.644033,1.079197
7,769500.0,27391.5,-14247,5,0,-2436.0,F,410579,247500.0,0,Y,Y,769500.0,0.026392,-8292,0.321637,0.332674,0.035596,3.109091,1.000000
8,284400.0,13387.5,-10971,,0,-426.0,M,105812,144000.0,0,N,Y,225000.0,0.010147,-6267,0.506329,0.304907,0.047073,1.975000,1.264000
9,284400.0,13387.5,-15615,1,0,-3459.0,F,127087,94500.0,0,Y,N,225000.0,0.031329,-4733,0.332278,0.376384,0.047073,3.009524,1.264000


In [40]:
new_df.dtypes



Amount_Credit               float64
Annuity                     float64
Birth_Day_CNT                 int64
Car_Age                      object
Default                       int64
Employed_Day_CNT            float64
Gender                       object
ID                            int64
Income                      float64
Num_of_Child                  int64
Own_Car                      object
Own_Residence                object
Price                       float64
Region_Population_Factor    float64
Registration_Day_CNT          int64
Income/Credit               float64
Annuity/Income              float64
Payment_rate                float64
Loan/Income                 float64
Credits/Goods               float64
dtype: object

In [41]:
credit.dtypes


Active_Credit      object
Credit            float64
Credit_Days         int64
Credit_Debt        object
Credit_Limit       object
Credit_Overdue    float64
Currency           object
File_ID             int64
ID                  int64
Max_Overdue        object
Overdue_Days        int64
Type               object
_id                object
dtype: object

In [42]:
#change data type for some variables from object to float
credit['Credit']=credit['Credit'].apply(conv_obj)
credit['Credit_Debt']=credit['Credit_Debt'].apply(conv_obj)
credit['Credit_Limit']=credit['Credit_Limit'].apply(conv_obj)
credit['Max_Overdue']=credit['Max_Overdue'].apply(conv_obj)
credit=credit.drop(columns=['_id'],axis=1)


In [43]:
credit.dtypes

Active_Credit      object
Credit            float64
Credit_Days         int64
Credit_Debt       float64
Credit_Limit      float64
Credit_Overdue    float64
Currency           object
File_ID             int64
ID                  int64
Max_Overdue       float64
Overdue_Days        int64
Type               object
dtype: object

In [44]:
#One_hot_encoding to deal with categorical variable
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object' and col!='_id']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)# extract value in each column
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
#summarize numerical variable
num_aggregations = {
    'Credit_Days': ['max', 'mean'],
    'Overdue_Days': ['max', 'mean'],
    'Max_Overdue': ['max', 'mean'],
    'Credit': ['median'],
    'Credit_Debt': ['mean'],
    'Credit_Overdue': ['max', 'mean']}
# summarize the new categorical variable using median
cat_aggregations = {}
new_bureau, bureau_cat = one_hot_encoder(credit, True)
for cat in bureau_cat: cat_aggregations[cat] = ['median']
    
# summarize information based on customer ID
bureau_agg = new_bureau.groupby('ID').agg({**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])


In [48]:
bureau_agg.head()

,BURO_Credit_Days_MAX,BURO_Credit_Days_MEAN,BURO_Overdue_Days_MAX,BURO_Overdue_Days_MEAN,BURO_Max_Overdue_MAX,BURO_Max_Overdue_MEAN,BURO_Credit_MEDIAN,BURO_Credit_Debt_MEAN,BURO_Credit_Overdue_MAX,BURO_Credit_Overdue_MEAN,...,BURO_Type_Cash loan (non-earmarked)_MEDIAN,BURO_Type_Consumer credit_MEDIAN,BURO_Type_Credit card_MEDIAN,BURO_Type_Loan for business development_MEDIAN,BURO_Type_Loan for working capital replenishment_MEDIAN,BURO_Type_Microloan_MEDIAN,BURO_Type_Mortgage_MEDIAN,BURO_Type_Real estate loan_MEDIAN,BURO_Type_Unknown type of loan_MEDIAN,BURO_Type_nan_MEDIAN
ID,,,,,,,,,,,,,,,,,,,,,
100016,-128,-677.833333,0,0.0,0.000,0.000000,73746.00,10620.750,0.0,0.0,...,0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
100053,-1764,-2287.714286,0,0.0,0.000,0.000000,77850.00,0.000,0.0,0.0,...,0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
100090,-315,-315.000000,0,0.0,0.000,0.000000,51957.00,21648.555,0.0,0.0,...,0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
100165,-1225,-1775.000000,0,0.0,0.000,0.000000,59034.24,0.000,0.0,0.0,...,0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0
100166,-106,-906.538462,0,0.0,9147.645,1050.220385,28476.00,8185.725,0.0,0.0,...,0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0


In [52]:
# find those records that credits are still active
active = new_bureau[new_bureau['Active_Credit_Active'] == 1]
active_agg = active.groupby('ID').agg(num_aggregations)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
active_agg.head()

,ACTIVE_Credit_Days_MAX,ACTIVE_Credit_Days_MEAN,ACTIVE_Overdue_Days_MAX,ACTIVE_Overdue_Days_MEAN,ACTIVE_Max_Overdue_MAX,ACTIVE_Max_Overdue_MEAN,ACTIVE_Credit_MEDIAN,ACTIVE_Credit_Debt_MEAN,ACTIVE_Credit_Overdue_MAX,ACTIVE_Credit_Overdue_MEAN
ID,,,,,,,,,,
100016,-128,-137.00,0,0.0,0.0,0.00,88996.5,21241.5000,0.0,0.0
100053,-2338,-2338.00,0,0.0,0.0,0.00,48820.5,0.0000,0.0,0.0
100090,-315,-315.00,0,0.0,0.0,0.00,51957.0,21648.5550,0.0,0.0
100166,-106,-206.00,0,0.0,0.0,0.00,86269.5,53207.2125,0.0,0.0
100245,-584,-910.75,0,0.0,1665.0,416.25,540000.0,259220.2500,0.0,0.0


In [54]:
#Reset index
bureau_agg=bureau_agg.reset_index()
active_agg=active_agg.reset_index()
#merge all credit records with active credit records
bureau_agg = bureau_agg.merge(active_agg, how='left', on='ID')
#one hot-encoding dealing with categorical variable in customer info table
for bin_feature in ['Gender', 'Own_Car']:
    new_df[bin_feature], uniques = pd.factorize(new_df[bin_feature])
new_df, cat_cols = one_hot_encoder(new_df, True)
#merge customer info with credit info
raw_agg = new_df.merge(bureau_agg, how='left', on='ID')
#fill na with 0 for some columns
raw_agg=raw_agg.fillna(0)
#get training data
train_df=raw_agg


In [58]:
import gc
folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=42)
#using stratifiedKFold to ensure different category's percentage in training and testing dataset the same as that in 
#original data

oof_preds = np.zeros(train_df.shape[0]) #get the number of rows in dataframe
feature_importance_df = pd.DataFrame() #get the importance for each variable
feats = [f for f in train_df.columns if f not in ['Default','ID','File_ID','index']] 
#drop columns that can't be used to build models
    
#split into training and validation dataset
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['Default'])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['Default'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['Default'].iloc[valid_idx]

#Use lightGBM to build model
    clf = LGBMClassifier(boosting_type='goss',nthread=4,n_estimators=2000,learning_rate=0.003,
        num_leaves=300,colsample_bytree=0.70,subsample=0.85,max_depth=10, reg_alpha=0.00,metric='binary_logloss',
        reg_lambda=0.00,min_child_weight=30,silent=-1,verbose=1)

    clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, early_stopping_rounds= 30)

    oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importances_
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
    gc.collect()
print('Full AUC score %.6f' % roc_auc_score(train_df['Default'], oof_preds))
print(feature_importance_df)

Training until validation scores don't improve for 30 rounds.
[100]	training's binary_logloss: 0.272998	training's auc: 0.73601	valid_1's binary_logloss: 0.275649	valid_1's auc: 0.665799
Early stopping, best iteration is:
[77]	training's binary_logloss: 0.274626	training's auc: 0.735752	valid_1's binary_logloss: 0.276697	valid_1's auc: 0.666488
Fold  1 AUC : 0.666488
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[37]	training's binary_logloss: 0.277744	training's auc: 0.730722	valid_1's binary_logloss: 0.278832	valid_1's auc: 0.663159
Fold  2 AUC : 0.663159
Training until validation scores don't improve for 30 rounds.
[100]	training's binary_logloss: 0.273294	training's auc: 0.733233	valid_1's binary_logloss: 0.275084	valid_1's auc: 0.6782
Early stopping, best iteration is:
[71]	training's binary_logloss: 0.275281	training's auc: 0.732872	valid_1's binary_logloss: 0.276519	valid_1's auc: 0.6792
Fold  3 AUC : 0.679200
Training until val

In [64]:
len(feats)

115

In [65]:
clf.feature_importances_

array([ 528,  875,  725, 1095,  284,  518,   70,  322,  699,  806,  884,
        516,  703, 1869,  468,  863,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,   60,    0,   45,   13,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])